In [5]:
import scanpy as sc

from scprint import scPrint
from scprint.tasks import Embedder, Denoiser, GNInfer

from scdataloader import Preprocessor
from scdataloader.utils import load_genes

import numpy as np
import pandas as pd

from collections import Counter
from scprint.model.utils import downsample_profile
from scipy.stats import spearmanr

from scdataloader import DataModule
from scprint.tasks import GNInfer
from anndata.utils import make_index_unique
from bengrn import BenGRN
import lamindb as ln
import json

%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
for n in ['adamson', 'nakatake', 'op_perturbation_sc_counts', 'replogle_K562_gwps_raw_singlecell', 'norman']:
    ln.Artifact.from_anndata(f"s3://openproblems-data/resources/grn/datasets_raw/{n}.h5ad", description=f"{n} geneRNIB dataset").save()

! no run & transform got linked, call `ln.track()` & re-run


In [107]:
2.0**adata.X.toarray()


AttributeError: '_CSRDataset' object has no attribute 'toarray'

In [55]:
adata = ln.Artifact.filter(description=f"op_perturbation_sc_counts geneRNIB dataset").one().open()
adata = adata[adata.obs['split']=='control']
adata = adata.to_memory()
if adata.layers is not None and 'counts' in adata.layers:
    adata.X = adata.layers['counts']
    del adata.layers['counts']
adata.write_h5ad('../data/op_perturbation_sc_counts.h5ad')

adata = ln.Artifact.filter(description=f"norman geneRNIB dataset").one().open()
adata = adata[adata.obs['control']==1]
adata = adata.to_memory()
if adata.layers is not None and 'counts' in adata.layers:
    adata.X = adata.layers['counts']
    del adata.layers['counts']
adata.write_h5ad('../data/norman.h5ad')



! run input wasn't tracked, call `ln.track()` and re-run


In [ ]:
aws s3 cp --no-sign-request s3://openproblems-data/resources/grn/grn_benchmark/prior/regulators_consensus_nakatake.json ../data/regulators_consensus_nakatake.json
aws s3 cp --no-sign-request s3://openproblems-data/resources/grn/grn_benchmark/evaluation_data/norman_bulk.h5ad ../data/norman_bulk.h5ad

aws s3 cp --no-sign-request s3://openproblems-data/resources/grn/grn_benchmark/prior/regulators_consensus_op.json ../data/regulators_consensus_op.json
aws s3 cp --no-sign-request s3://openproblems-data/resources/grn/grn_benchmark/evaluation_data/op_bulk.h5ad ../data/op_bulk.h5ad

aws s3 cp --no-sign-request s3://openproblems-data/resources/grn/grn_benchmark/prior/regulators_consensus_norman.json ../data/regulators_consensus_norman.json

In [117]:
FOLDER = '/home/ml4ig1/task_grn_inference/resources/grn_benchmark/'
adata = sc.read_h5ad(FOLDER+'evaluation_data/op_bulk.h5ad')
reg = json.load(open(FOLDER+'prior/regulators_consensus_op.json'))

In [118]:
n_features_theta_min = np.asarray(
    [reg[gene_name]["0"] for gene_name in adata.var_names], dtype=int
)
n_features_theta_median = np.asarray(
    [reg[gene_name]["0.5"] for gene_name in adata.var_names], dtype=int
)
n_features_theta_max = np.asarray(
    [reg[gene_name]["1"] for gene_name in adata.var_names], dtype=int
)

In [119]:
adata.var["n_features_theta_"+str(0.0)] = n_features_theta_min
adata.var["n_features_theta_"+str(0.5)] = n_features_theta_median
adata.var["n_features_theta_"+str(1.0)] = n_features_theta_max

In [120]:
adata.write_h5ad('../../benGRN/data/replogle_rna_test.h5ad')